Batch Normalization

In [ ]:
# Import modules

import tensorflow as tf
import tensorflow.keras.layers as layers


In [ ]:
net=layers.BatchNormalization()
x=tf.random.normal([2,3])
out=net(x)

print(net.trainable_variables)
print("\n")
print(net.variables)

[<tf.Variable 'batch_normalization_3/gamma:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>, <tf.Variable 'batch_normalization_3/beta:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>]


[<tf.Variable 'batch_normalization_3/gamma:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>, <tf.Variable 'batch_normalization_3/beta:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>, <tf.Variable 'batch_normalization_3/moving_mean:0' shape=(3,) dtype=float32, numpy=array([0., 0., 0.], dtype=float32)>, <tf.Variable 'batch_normalization_3/moving_variance:0' shape=(3,) dtype=float32, numpy=array([1., 1., 1.], dtype=float32)>]


Example with non-zero mean

In [ ]:
x=tf.random.normal([2,3,3,4],mean=1,stddev=0.5)
net = layers.BatchNormalization(axis=3)
out=net(x)

net.variables

[<tf.Variable 'batch_normalization_4/gamma:0' shape=(4,) dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization_4/beta:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_4/moving_mean:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_4/moving_variance:0' shape=(4,) dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>]

Forward update

In [ ]:
x=tf.random.normal([2,3,3,4],mean=1,stddev=0.5)
net = layers.BatchNormalization(axis=3)
out=net(x,training=True)
print(net.variables)

print("\n\n","Training","\n\n")
for i in range(100):
  out=net(x,training=True)
# Now moving_mean should be closer to 1 and moving variance closer to 0.25 (stddev root of variance)
net.variables

[<tf.Variable 'batch_normalization_6/gamma:0' shape=(4,) dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>, <tf.Variable 'batch_normalization_6/beta:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>, <tf.Variable 'batch_normalization_6/moving_mean:0' shape=(4,) dtype=float32, numpy=array([0.00846255, 0.01142006, 0.00966587, 0.00950637], dtype=float32)>, <tf.Variable 'batch_normalization_6/moving_variance:0' shape=(4,) dtype=float32, numpy=array([0.9938263, 0.9919904, 0.9924904, 0.9927167], dtype=float32)>]


 Training 




[<tf.Variable 'batch_normalization_6/gamma:0' shape=(4,) dtype=float32, numpy=array([1., 1., 1., 1.], dtype=float32)>,
 <tf.Variable 'batch_normalization_6/beta:0' shape=(4,) dtype=float32, numpy=array([0., 0., 0., 0.], dtype=float32)>,
 <tf.Variable 'batch_normalization_6/moving_mean:0' shape=(4,) dtype=float32, numpy=array([0.539596  , 0.72817534, 0.6163227 , 0.6061525 ], dtype=float32)>,
 <tf.Variable 'batch_normalization_6/moving_variance:0' shape=(4,) dtype=float32, numpy=array([0.6063493 , 0.48928437, 0.5211675 , 0.5355983 ], dtype=float32)>]

Backward update

In [ ]:
from tensorflow.keras import optimizers
# Optimizer just aken from Cifar100 example
optimizer=optimizers.Adam(lr=1e-4)
for i in range(10):
  with tf.GradientTape() as tape:
    out=net(x,training=True)
    loss=tf.reduce_mean(tf.pow(out,2))-1
  grads=tape.gradient(loss,net.trainable_variables)
  optimizer.apply_gradients(zip(grads,net.trainable_variables))

#Slight changes in gamma and beta -> Modifies original model to fit data as if it was a normal distribution
net.variables

/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


[<tf.Variable 'batch_normalization_6/gamma:0' shape=(4,) dtype=float32, numpy=array([0.99799967, 0.99799967, 0.99799967, 0.99799967], dtype=float32)>,
 <tf.Variable 'batch_normalization_6/beta:0' shape=(4,) dtype=float32, numpy=
 array([-2.3350294e-05,  1.3605357e-05, -1.3338593e-05,  8.9839905e-06],
       dtype=float32)>,
 <tf.Variable 'batch_normalization_6/moving_mean:0' shape=(4,) dtype=float32, numpy=array([0.60042787, 0.8102671 , 0.685804  , 0.6744873 ], dtype=float32)>,
 <tf.Variable 'batch_normalization_6/moving_variance:0' shape=(4,) dtype=float32, numpy=array([0.5619707 , 0.43170843, 0.46718583, 0.48324347], dtype=float32)>]